Домашку будет легче делать в колабе (убедитесь, что у вас runtype с gpu).

# Задание 1 (3 балла)

Обучите word2vec модели с негативным семплированием (cbow и skip-gram) аналогично тому, как это было сделано в семинаре. Вам нужно изменить следующие пункты:
1) добавьте лемматизацию в предобработку (любым способом)  
2) измените размер окна в большую или меньшую сторону
3) измените размерность итоговых векторов

Выберете несколько не похожих по смыслу слов (не таких как в семинаре), и протестируйте полученные эмбединги (найдите ближайшие слова и оцените качество, как в семинаре).
Постарайтесь обучать модели как можно дольше и на как можно большем количестве данных. (Но если у вас мало времени или ресурсов, то допустимо взять поменьше данных и поставить меньше эпох)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install razdel
!pip install pymorphy3
!pip install regex

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 70.6 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import razdel
import pymorphy3
import regex as re
from string import punctuation
from sklearn.model_selection import train_test_split
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_distances

from IPython.display import Image
from IPython.core.display import HTML
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
print("Torch version:", torch.__version__)

Using device: cpu
Torch version: 2.9.0+cpu


In [5]:
wiki = open('sample_data/wiki_data.txt').read().split('\n')
len(wiki)

20003

In [6]:
morph = pymorphy3.MorphAnalyzer()

def preprocess_with_lemma(text):
    text = text.lower()
    tokens = [token.text for token in razdel.tokenize(text)]

    lemmatized_tokens = []
    for token in tokens:
        clean_token = re.sub(r'[^а-яё]', '', token)
        if clean_token:
            lemma = morph.parse(clean_token)[0].normal_form
            lemmatized_tokens.append(lemma)

    return lemmatized_tokens

lemmas = [preprocess_with_lemma(sent) for sent in wiki]

In [7]:
vocab = Counter()
for sent in lemmas:
    vocab.update(sent)

filtered_vocab = set()
for word in vocab:
    if vocab[word] > 30:
        filtered_vocab.add(word)

print(len(filtered_vocab))
print(list(filtered_vocab)[:4])

11693
['усик', 'афины', 'отделиться', 'сандра']


In [ ]:
word2id = {'UNK': 0}
for word in filtered_vocab:
    word2id[word] = len(word2id)

id2word = {i: word for word, i in word2id.items()}

indexed_wiki = []
for sent in lemmas:
    indexed_sent = [word2id.get(word, word2id['UNK']) for word in sent]
    indexed_wiki.append(indexed_sent)

print(len(indexed_wiki))
print(indexed_wiki[:10])

20003
[[0, 3796, 4449, 0, 9197, 8360, 11201, 6714, 11671, 3796, 4449, 4908, 11201, 9826, 0, 8589, 8360, 3529, 11201, 5103, 6990, 11202, 1737, 7573, 0, 6130, 5103, 6990, 10169, 1737, 11076, 0, 3114, 3014, 7313, 4966, 0, 4396, 11254, 4966, 8099, 5615, 5724, 5919, 6726, 0, 750, 5936, 8360, 0, 5103, 6130, 6235, 0, 750, 5936, 8360, 8099, 5103, 8000, 0, 8273, 0, 4297, 2922, 11464, 6793, 5656, 578, 8000, 0, 2385, 7315, 8575, 3114, 0, 11065, 6793, 8000, 9781, 0, 3209, 11201, 7903, 6271, 878, 7497, 4089, 7573, 0, 6130, 0, 3529, 1219, 11201, 6130, 5103, 6990, 10169, 11201, 8025, 11671, 4746, 11201, 8360, 1127, 7115, 9132, 8201, 1975, 11201, 8360, 8099, 368, 8360, 3529, 1295, 4037, 8547, 2085, 0, 746, 6130, 0, 6578, 11201, 4321, 1217, 5423, 2243, 2168, 331, 6271, 8360, 0, 2639, 9667, 0, 5687, 9241, 10071, 6130, 4192, 5931, 8729, 7497, 2658, 6130, 0, 2555, 0, 8619, 11201, 6271, 11201, 8360, 9241, 10071, 6130, 4192, 7497, 10091, 11596, 11072, 3114, 212, 6271, 11201, 8360, 8893, 10071, 6130, 4192, 1

cbow

In [ ]:
def gen_batches_cbow_neg(indexed_wiki, window_size=3, batch_size=512, n_neg=5):
    X_context = []
    y_target = []
    labels = []

    for sent in indexed_wiki:
        for i in range(len(sent)):
            start = max(0, i - window_size)
            end = min(len(sent), i + window_size + 1)

            context = sent[start:i] + sent[i+1:end]

            if len(context) < window_size * 2:
                continue

            X_context.append(context)
            y_target.append(sent[i])
            labels.append(1)

            for _ in range(n_neg):
                X_context.append(context)
                y_target.append(np.random.randint(0, len(word2id)))
                labels.append(0)

            if len(X_context) >= batch_size:
                yield (torch.LongTensor(X_context),
                       torch.LongTensor(y_target),
                       torch.FloatTensor(labels))
                X_context, y_target, labels = [], [], []


In [ ]:
class CBOWModelNeg(nn.Module):
    def __init__(self, vocab_size, emb_dim=256):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, emb_dim)
        self.out_embeddings = nn.Embedding(vocab_size, emb_dim)

    def forward(self, context, target):
        context_emb = self.embeddings(context).mean(dim=1)
        target_emb = self.out_embeddings(target)

        scores = (context_emb * target_emb).sum(dim=1)
        return scores

In [ ]:
EMB_DIM = 256
WINDOW_SIZE = 3
VOCAB_SIZE = len(word2id)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

cbow_model = CBOWModelNeg(VOCAB_SIZE, EMB_DIM).to(DEVICE)
optimizer = torch.optim.Adam(cbow_model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()

In [ ]:
for epoch in range(5):
    losses = []

    for x, y, labels in gen_batches_cbow_neg(indexed_wiki, window_size=WINDOW_SIZE, batch_size=512):
        x, y, labels = x.to(DEVICE), y.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = cbow_model(x, y)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        if len(losses) % 500 == 0:
            print(f"CBOW | Epoch {epoch+1} | Batch {len(losses)} | Loss: {np.mean(losses[-500:]):.4f}")

    print(f"CBOW Epoch {epoch+1}. Средний Loss: {np.mean(losses):.4f}")

cbow_weights = cbow_model.embeddings.weight.detach().cpu().numpy()

CBOW | Epoch 1 | Batch 500 | Loss: 2.5747
CBOW | Epoch 1 | Batch 1000 | Loss: 2.2913
CBOW | Epoch 1 | Batch 1500 | Loss: 1.9957
CBOW | Epoch 1 | Batch 2000 | Loss: 1.7838
CBOW | Epoch 1 | Batch 2500 | Loss: 1.5694
CBOW | Epoch 1 | Batch 3000 | Loss: 1.3176
CBOW | Epoch 1 | Batch 3500 | Loss: 1.1676
CBOW | Epoch 1 | Batch 4000 | Loss: 1.0492
CBOW | Epoch 1 | Batch 4500 | Loss: 0.9737
CBOW | Epoch 1 | Batch 5000 | Loss: 0.9472
CBOW | Epoch 1 | Batch 5500 | Loss: 0.9390
CBOW | Epoch 1 | Batch 6000 | Loss: 0.7155
CBOW | Epoch 1 | Batch 6500 | Loss: 0.8449
CBOW | Epoch 1 | Batch 7000 | Loss: 0.8173
CBOW | Epoch 1 | Batch 7500 | Loss: 0.8422
CBOW | Epoch 1 | Batch 8000 | Loss: 0.7902
CBOW | Epoch 1 | Batch 8500 | Loss: 0.7830
CBOW | Epoch 1 | Batch 9000 | Loss: 0.7791
CBOW | Epoch 1 | Batch 9500 | Loss: 0.7212
CBOW | Epoch 1 | Batch 10000 | Loss: 0.7292
CBOW | Epoch 1 | Batch 10500 | Loss: 0.7312
CBOW | Epoch 1 | Batch 11000 | Loss: 0.7259
CBOW | Epoch 1 | Batch 11500 | Loss: 0.7141
CBOW | E

In [ ]:
torch.save(cbow_model.state_dict(), '/content/drive/My Drive/cbow_model.pth')

In [ ]:
def get_closest_words(word, weights, word2id, id2word, top_n=5):
    if word not in word2id:
        return "Слова нет в словаре"

    word_id = word2id[word]
    vec = weights[word_id]

    norms = np.linalg.norm(weights, axis=1)
    cos_sim = np.dot(weights, vec) / (norms * np.linalg.norm(vec))

    closest_ids = np.argsort(cos_sim)[-(top_n+1):-1][::-1]

    return [id2word[idx] for idx in closest_ids]

skipgram

In [ ]:
def gen_batches_sg_neg(indexed_wiki, window_size=3, batch_size=512, n_neg=5):
    X_target = []
    y_context = []
    labels = []

    for sent in indexed_wiki:
        for i in range(len(sent)):
            start = max(0, i - window_size)
            end = min(len(sent), i + window_size + 1)
            context = sent[start:i] + sent[i+1:end]

            for context_word in context:
                X_target.append(sent[i])
                y_context.append(context_word)
                labels.append(1)

                for _ in range(n_neg):
                    X_target.append(sent[i])
                    y_context.append(np.random.randint(0, len(word2id)))
                    labels.append(0)

                if len(X_target) >= batch_size:
                    yield (torch.LongTensor(X_target),
                           torch.LongTensor(y_context),
                           torch.FloatTensor(labels))
                    X_target, y_context, labels = [], [], []

In [ ]:
class SkipGramModelNeg(nn.Module):
    def __init__(self, vocab_size, emb_dim=256):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, emb_dim)
        self.out_embeddings = nn.Embedding(vocab_size, emb_dim)

    def forward(self, target, context):
        target_emb = self.embeddings(target)
        context_emb = self.out_embeddings(context)

        scores = (target_emb * context_emb).sum(dim=1)
        return scores

In [ ]:
EMB_DIM = 256
WINDOW_SIZE = 3
VOCAB_SIZE = len(word2id)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

sg_model = SkipGramModelNeg(VOCAB_SIZE, EMB_DIM).to(DEVICE)
optimizer = torch.optim.Adam(sg_model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()

for epoch in range(5):
    losses = []

    for x, y, labels in gen_batches_sg_neg(indexed_wiki, window_size=WINDOW_SIZE, batch_size=512):
        x, y, labels = x.to(DEVICE), y.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = sg_model(x, y)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        if len(losses) % 500 == 0:
            print(f"Skip-gram | Epoch {epoch+1} | Batch {len(losses)} | Loss: {np.mean(losses[-500:]):.4f}")

    print(f"Skip-gram Epoch {epoch+1}. Средний Loss: {np.mean(losses):.4f}")

sg_weights = sg_model.embeddings.weight.detach().cpu().numpy()

Skip-gram | Epoch 1 | Batch 500 | Loss: 6.2129
Skip-gram | Epoch 1 | Batch 1000 | Loss: 5.7592
Skip-gram | Epoch 1 | Batch 1500 | Loss: 5.4359
Skip-gram | Epoch 1 | Batch 2000 | Loss: 5.1119
Skip-gram | Epoch 1 | Batch 2500 | Loss: 4.8331
Skip-gram | Epoch 1 | Batch 3000 | Loss: 4.7254
Skip-gram | Epoch 1 | Batch 3500 | Loss: 4.7766
Skip-gram | Epoch 1 | Batch 4000 | Loss: 4.5209
Skip-gram | Epoch 1 | Batch 4500 | Loss: 4.2831
Skip-gram | Epoch 1 | Batch 5000 | Loss: 4.1786
Skip-gram | Epoch 1 | Batch 5500 | Loss: 4.1382
Skip-gram | Epoch 1 | Batch 6000 | Loss: 3.9299
Skip-gram | Epoch 1 | Batch 6500 | Loss: 3.8407
Skip-gram | Epoch 1 | Batch 7000 | Loss: 3.5969
Skip-gram | Epoch 1 | Batch 7500 | Loss: 3.5200
Skip-gram | Epoch 1 | Batch 8000 | Loss: 3.4420
Skip-gram | Epoch 1 | Batch 8500 | Loss: 3.3999
Skip-gram | Epoch 1 | Batch 9000 | Loss: 3.2607
Skip-gram | Epoch 1 | Batch 9500 | Loss: 3.2308
Skip-gram | Epoch 1 | Batch 10000 | Loss: 3.1135
Skip-gram | Epoch 1 | Batch 10500 | Loss

In [ ]:
torch.save(sg_model.state_dict(), '/content/drive/My Drive/sg_model_weights.pth')

In [ ]:
test_words = ['новостройка', 'река', 'посёлок', 'фильм', 'область']

for word in test_words:
    closest = get_closest_words(word, sg_weights, word2id, id2word)
    print(f"Слово: {word} | Соседи: {closest}")

Слово: новостройка | Соседи: Слова нет в словаре
Слово: река | Соседи: ['берег', 'озеро', 'приток', 'долина', 'км']
Слово: посёлок | Соседи: ['село', 'поселение', 'район', 'сельский', 'деревня']
Слово: фильм | Соседи: ['режиссёр', 'сериал', 'картина', 'роман', 'эпизод']
Слово: область | Соседи: ['район', 'ныне', 'сельский', 'ростовский', 'губерния']


In [ ]:
test_words = ['новостройка', 'река', 'посёлок', 'фильм', 'область']

for word in test_words:
    closest = get_closest_words(word, cbow_weights, word2id, id2word)
    print(f"Слово: {word} | Соседи: {closest}")

Слово: новостройка | Соседи: Слова нет в словаре
Слово: река | Соседи: ['берег', 'озеро', 'ручей', 'долина', 'км']
Слово: посёлок | Соседи: ['село', 'район', 'хутор', 'деревня', 'улица']
Слово: фильм | Соседи: ['сериал', 'мультфильм', 'эпизод', 'режиссёр', 'роман']
Слово: область | Соседи: ['губерния', 'уезд', 'район', 'областной', 'край']


Удивительно, но сильно дольше обучавшийся skip-gram в итоге справился не сильно лучше cbow модели. Больше всего промахов у модели skip-gram с подбором синонимов к слову "область", тут и "ныне" и "сельский" довольно далеки.

# Задание 2 (2 балла)

Обучите 1 word2vec и 1 fastext модель в gensim. В каждой из модели нужно задать все параметры, которые мы разбирали на семинаре. Заданные значения должны отличаться от дефолтных и от тех, что мы использовали на семинаре.

In [10]:
from gensim.models import Word2Vec, FastText

In [11]:
model_w2v = Word2Vec(sentences=lemmas,
                     vector_size=256,
                     window=5,
                     min_count=5,
                     sg=1,
                     epochs=5)

In [16]:
print(model_w2v.wv.most_similar('каша'))

[('взбить', 0.870908796787262), ('ячменный', 0.8703863024711609), ('жареный', 0.8668773174285889), ('обжарить', 0.8611367344856262), ('бульон', 0.8577075600624084), ('сметана', 0.857672929763794), ('приправа', 0.8549471497535706), ('набитый', 0.8544008731842041), ('пирожное', 0.8543984293937683), ('репчатый', 0.8529167771339417)]


In [17]:
model_ft = FastText(sentences=lemmas,
                    vector_size=256,
                    window=5,
                    min_count=5,
                    sg=1,
                    epochs=5)

In [18]:
print(model_ft.wv.most_similar('каша'))

[('кашар', 0.888312816619873), ('аша', 0.7705908417701721), ('паприкаша', 0.7528278827667236), ('кашкра', 0.7398574352264404), ('чаша', 0.7350392937660217), ('паша', 0.7347075939178467), ('кабаниха', 0.7147276997566223), ('каштан', 0.7145907878875732), ('камыш', 0.7047783732414246), ('даша', 0.7017403841018677)]


# Задание 3 (3 балла)

Используя датасет для классификации (labeled.csv), обучите классификатор на базе эмбеддингов. Оцените качество на отложенной выборке.   
В качестве эмбеддинг модели вы можете использовать одну из моделей обученных в предыдущем задании или использовать одну из предобученных моделей с rusvectores (удостоверьтесь что правильно воспроизводите предобработку в этом случае!)  
Для того, чтобы построить эмбединг целого текста, усредните вектора отдельных слов в один общий вектор.
В качестве алгоритма классификации используйте LogisicticRegression (можете попробовать SGDClassifier, чтобы было побыстрее)  
F1 мера должна быть выше 20%.

In [22]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np

In [23]:
data = pd.read_csv('sample_data/labeled.csv')

In [24]:
data['norm_text'] = data['comment'].apply(preprocess_with_lemma)

In [25]:
def get_embedding(text, model, vector_size):
    vectors = [model_w2v.wv[word] for word in text if word in model_w2v.wv]

    if not vectors:
        return np.zeros(vector_size)

    return np.mean(vectors, axis=0)

DIM = 256
X = np.array([get_embedding(text, model_w2v, DIM) for text in data['norm_text']])
y = data['toxic'].values

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = LogisticRegression(C=1.0, max_iter=1000)
clf.fit(X_train, y_train)

preds = clf.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.84      0.92      0.87      1944
         1.0       0.78      0.63      0.70       939

    accuracy                           0.82      2883
   macro avg       0.81      0.77      0.79      2883
weighted avg       0.82      0.82      0.82      2883



# Задание 4 (2 доп балла)

В тетрадку с фастекстом добавьте код для обучения с negative sampling (задача сводится к бинарной классификации) и обучите модель. Проверьте полученную модель на нескольких словах. Похожие слова должны быть похожими по смыслу и по форме.

In [29]:
import numpy as np
import pandas as pd
from string import punctuation
from sklearn.model_selection import train_test_split
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_distances

from IPython.display import Image
from IPython.core.display import HTML
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
print("Torch version:", torch.__version__)

Using device: cpu
Torch version: 2.9.0+cpu


In [51]:
import re
from collections import Counter
def tokenize(text):
    tokens = re.sub('#+', ' ', text.lower()).split()
    tokens = [token.strip(punctuation) for token in tokens]
    tokens = [token for token in tokens if token]
    return tokens

In [53]:
def ngrammer(raw_string, n=2):
    ngrams = []
    raw_string = ''.join(['<', raw_string, '>'])
    for i in range(0,len(raw_string)-n+1):
        ngram = ''.join(raw_string[i:i+n])
        if ngram == '<' or ngram == '>':
            continue
        ngrams.append(ngram)
    return ngrams

In [54]:
def split_tokens(tokens, min_ngram_size, max_ngram_size):
    tokens_with_subwords = []
    for token in tokens:
        subtokens = []
        for i in range(min_ngram_size, max_ngram_size+1):
            if len(token) > i:
                subtokens.extend(ngrammer(token, i))
        tokens_with_subwords.append(subtokens)
    return tokens_with_subwords

In [55]:
class SubwordTokenizer:
    def __init__(self, ngram_range=(1,1), min_count=5):
        self.min_ngram_size, self.max_ngram_size = ngram_range
        self.min_count = min_count
        self.subword_vocab = None
        self.fullword_vocab = None
        self.vocab = None
        self.id2word = None
        self.word2id = None

    def build_vocab(self, texts):
        # чтобы построить словарь нужно пройти по всему корпусу и собрать частоты всех уникальных слов и нграммов
        unfiltered_subword_vocab = Counter()
        unfiltered_fullword_vocab = Counter()
        for text in texts:
            tokens = tokenize(text)
            unfiltered_fullword_vocab.update(tokens)
            subwords_per_token = split_tokens(tokens, self.min_ngram_size, self.max_ngram_size)
            for subwords in subwords_per_token:
                # в одном слове могут быть одинаковые нграммы поэтому возьмем только уникальные
                unfiltered_subword_vocab.update(set(subwords))

        self.fullword_vocab = set()
        self.subword_vocab = set()

        # теперь отфильтруем по частоте
        for word, count in unfiltered_fullword_vocab.items():
            if count >= self.min_count:
                self.fullword_vocab.add(word)
        # для нграммов сделаем порог побольше чтобы не создавать слишком много нграммов
        # и учитывать только действительно частотные
        for word, count in unfiltered_subword_vocab.items():
            if count >= (self.min_count * 100):
                self.subword_vocab.add(word)

        # общий словарь
        self.vocab = self.fullword_vocab | self.subword_vocab
        self.id2word = {i:word for i,word in enumerate(self.vocab)}
        self.word2id = {word:i for i,word in self.id2word.items()}

    def subword_tokenize(self, text):
        if self.vocab is None:
            raise AttributeError('Vocabulary is not built!')
        # разбиваем на токены
        tokens = tokenize(text)
        # каждый токен разбиваем на символьные нграммы
        tokens_with_subwords = split_tokens(tokens, self.min_ngram_size, self.max_ngram_size)
        # оставляет только токены и нграммы которые есть в словаре
        only_vocab_tokens_with_subwords = []
        for full_token, sub_tokens in zip(tokens, tokens_with_subwords):
            filtered = []
            if full_token in self.vocab:
                # само слово и нграммы хранятся в одном списке
                # но слово будет всегда первым в списке
                filtered.append(full_token)
            filtered.extend([subtoken for subtoken in set(sub_tokens) if subtoken in self.vocab])
            only_vocab_tokens_with_subwords.append(filtered)

        return only_vocab_tokens_with_subwords

    def encode(self, subword_tokenized_text):
        # маппим токены и нграммы в их индексы в словаре
        encoded_text = []
        for token in subword_tokenized_text:
            if not token:
                continue
            encoded_text.append([self.word2id[token[0]]] + [self.word2id[t] for t in set(token[1:]) if t in self.word2id and t != token[0]])
        return encoded_text

    def __call__(self, text):
        return self.encode(self.subword_tokenize(text))

In [56]:
tokenizer = SubwordTokenizer(ngram_range=(2,4), min_count=10)
tokenizer.build_vocab(wiki)

In [57]:
class FastTextNegSampling(nn.Module):
    def __init__(self, vocab_size, emb_dim=100):
        super().__init__()
        self.target_embeddings = nn.EmbeddingBag(vocab_size, emb_dim, mode='mean')
        self.context_embeddings = nn.Embedding(vocab_size, emb_dim)

    def forward(self, target_subwords, context_words, offsets):
        target_vecs = self.target_embeddings(target_subwords, offsets)
        context_vecs = self.context_embeddings(context_words)

        return torch.sum(target_vecs * context_vecs, dim=1)

In [58]:
def gen_batches_neg(sentences, tokenizer, window=5, batch_size=512, n_neg=5):
    while True:
        target_subwords, offsets, contexts, labels = [], [0], [], []

        for sent in sentences:
            encoded = tokenizer(sent)
            if len(encoded) < 2: continue

            for i in range(len(encoded)):
                target = encoded[i]
                start = max(0, i - window)
                end = min(len(encoded), i + window + 1)
                pos_indices = [encoded[j][0] for j in range(start, end) if i != j]

                for pos_idx in pos_indices:
                    target_subwords.extend(target)
                    offsets.append(offsets[-1] + len(target))
                    contexts.append(pos_idx)
                    labels.append(1)

                    for _ in range(n_neg):
                        target_subwords.extend(target)
                        offsets.append(offsets[-1] + len(target))
                        contexts.append(np.random.randint(0, len(tokenizer.vocab)))
                        labels.append(0)

                if len(contexts) >= batch_size:
                    yield (torch.LongTensor(target_subwords),
                           torch.LongTensor(offsets[:-1]),
                           torch.LongTensor(contexts),
                           torch.FloatTensor(labels))
                    target_subwords, offsets, contexts, labels = [], [0], [], []

In [59]:
model = FastTextNegSampling(len(tokenizer.vocab)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()

train_gen = gen_batches_neg(wiki, tokenizer, batch_size=1024)

for step in range(2001):
    subwords, off, ctx, lbl = next(train_gen)

    optimizer.zero_grad()
    preds = model(subwords.to(device), ctx.to(device), off.to(device))
    loss = criterion(preds, lbl.to(device))
    loss.backward()
    optimizer.step()

    if step % 500 == 0:
        print(f"Step {step}, Loss: {loss.item():.4f}")

Step 0, Loss: 1.5416
Step 500, Loss: 1.6266
Step 1000, Loss: 1.3166
Step 1500, Loss: 1.6304
Step 2000, Loss: 1.1620


In [60]:
def get_vector(word, model, tokenizer):
    model.eval()
    tokens = tokenizer(word)[0]
    subwords = torch.LongTensor(tokens).to(device)
    offset = torch.LongTensor([0]).to(device)
    with torch.no_grad():
        vec = model.target_embeddings(subwords, offset)
    return vec.cpu().numpy()

In [64]:
get_vector("нос", model, tokenizer)

array([[ 0.13846257, -0.14302301,  0.15178375,  0.6185609 ,  0.32083163,
         0.1683947 , -0.47352323,  0.16885343, -0.09866892,  0.3684754 ,
         0.33515102,  0.16406013, -0.37696043, -0.06562246,  0.4532543 ,
         0.21261689,  0.20693934, -0.00247377, -0.50505364,  0.43035278,
         0.75401926,  0.15918767,  1.0646186 , -0.50896984,  0.28520927,
        -0.24431667,  0.38205248, -0.05343943,  0.00879141,  0.13118945,
        -0.3463318 ,  0.63581896,  0.2885397 ,  0.9826933 ,  0.08578312,
        -0.32318664,  0.33146152,  0.46398038,  0.34402713, -0.12774496,
        -0.26661348,  1.174057  , -0.17074664,  0.07546492,  0.16881272,
         0.33689183, -0.6178116 ,  0.15365107,  0.33820468, -0.6467811 ,
        -0.55077016,  0.28717497,  0.40538654,  0.6012375 , -0.71409583,
        -0.08945696,  0.5198597 , -0.8568907 ,  0.4533511 , -0.60046464,
         0.36029738, -0.06455822,  0.34550548,  0.23540969,  0.27582708,
         0.47075024,  0.29856104,  0.14271839,  0.0

In [65]:
def most_similar_neg(word, model, tokenizer, top_n=10):
    word_vec = get_vector(word, model, tokenizer)

    all_embeddings = model.target_embeddings.weight.detach().cpu().numpy()

    distances = cosine_distances(word_vec.reshape(1, -1), all_embeddings)[0]

    indices = distances.argsort()[:top_n]

    return [(tokenizer.id2word[i], distances[i]) for i in indices]

print(most_similar_neg("нос", model, tokenizer))

[('нос', np.float32(0.47815406)), ('ос', np.float32(0.49888468)), ('с>', np.float32(0.5321866)), ('сас', np.float32(0.60100335)), ('собирают', np.float32(0.60179764)), ('прошлом', np.float32(0.6234371)), ('насколько', np.float32(0.62434417)), ('<н', np.float32(0.628259)), ('заблуждение', np.float32(0.6288296)), ('сох', np.float32(0.6330385))]
